# **Yelp Rating Predictor**

Objective: predict Yelp! rating for newly opened bakery based on the information of other restaurants' rating provided by Yelp!

Use Multiple Lineal Regression model to decide what factors contribute the most to a high rating



## Import data and clean

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import pandas as pd

In [19]:
#import raw data from folders and convert to pandas df

businesses = pd.read_json('raw_data/yelp_business.json',lines=True)
reviews = pd.read_json('raw_data/yelp_review.json',lines=True)
users = pd.read_json('raw_data/yelp_user.json',lines=True)
checkins = pd.read_json('raw_data/yelp_checkin.json',lines=True)
tips = pd.read_json('raw_data/yelp_tip.json',lines=True)
photos = pd.read_json('raw_data/yelp_photo.json',lines=True)

In [20]:
businesses.head()

,address,alcohol?,attributes,business_id,categories,city,good_for_kids,has_bike_parking,has_wifi,hours,...,longitude,name,neighborhood,postal_code,price_range,review_count,stars,state,take_reservations,takes_credit_cards
0,1314 44 Avenue NE,0,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,1,0,0,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",...,-114.031675,Minhas Micro Brewery,,T2E 6L6,2,24,4.0,AB,1,1
1,,0,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,1,0,0,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",...,-114.939821,CK'S BBQ & Catering,,89002,2,3,4.5,NV,0,1
2,1335 rue Beaubien E,1,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,1,1,1,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",...,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,2,5,4.0,QC,1,0
3,211 W Monroe St,0,None,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Phoenix,0,0,0,None,...,-112.076979,Geico Insurance,,85003,0,8,1.5,AZ,0,0
4,2005 Alyth Place SE,0,{'BusinessAcceptsCreditCards': 'True'},8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,0,0,0,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",...,-114.027366,Action Engine,,T2H 0N5,0,4,2.0,AB,0,1


In [21]:
reviews.head()

,business_id,average_review_age,average_review_length,average_review_sentiment,number_funny_votes,number_cool_votes,number_useful_votes
0,--1UhMGODdWsrMastO9DZw,524.458333,466.208333,0.808638,1,16,15
1,--6MefnULPED_I942VcFNA,1199.589744,785.205128,0.669126,27,32,53
2,--7zmmkVg-IMGaXbuVd0SQ,717.851852,536.592593,0.820837,29,52,81
3,--8LPVSo5i0Oo61X01sV9A,751.750000,478.250000,0.170925,0,0,9
4,--9QQLMTbFzLJ_oT-ON3Xw,978.727273,436.181818,0.562264,3,4,7


In [16]:
print(reviews.shape)
print(businesses.shape)
print(checkins.shape)
print(photos.shape)
print(tips.shape)
print(users.shape)

(188593, 7)
(188593, 22)
(157075, 4)
(32976, 3)
(121526, 3)
(188593, 6)


In [22]:
checkins.head()

,business_id,time,weekday_checkins,weekend_checkins
0,7KPBkxAOEtb3QeIL9PEErg,"{'Fri-0': 2, 'Sat-0': 1, 'Sun-0': 1, 'Wed-0': ...",76,75
1,kREVIrSBbtqBhIYkTccQUg,"{'Mon-13': 1, 'Thu-13': 1, 'Sat-16': 1, 'Wed-1...",4,3
2,tJRDll5yqpZwehenzE2cSg,"{'Thu-0': 1, 'Mon-1': 1, 'Mon-12': 1, 'Sat-16'...",3,3
3,tZccfdl6JNw-j5BKnCTIQQ,"{'Sun-14': 1, 'Fri-18': 1, 'Mon-20': 1}",1,2
4,r1p7RAMzCV_6NPF0dNoR3g,"{'Sat-3': 1, 'Sun-18': 1, 'Sat-21': 1, 'Sat-23...",1,4


In [23]:
photos.head()

,business_id,average_caption_length,number_pics
0,--1UhMGODdWsrMastO9DZw,0.000000,1
1,--6MefnULPED_I942VcFNA,67.500000,2
2,--9e1ONYQuAa-CB_Rrw7Tw,30.426471,136
3,--DaPTJW3-tB1vP-PfdTEg,0.000000,1
4,--FBCX-N37CMYDfs790Bnw,5.500000,4


In [24]:
users.head()

,business_id,average_number_friends,average_days_on_yelp,average_number_fans,average_review_count,average_number_years_elite
0,--1UhMGODdWsrMastO9DZw,18.791667,1789.750000,1.833333,57.541667,0.833333
1,--6MefnULPED_I942VcFNA,214.564103,2039.948718,49.256410,332.743590,1.769231
2,--7zmmkVg-IMGaXbuVd0SQ,126.185185,1992.796296,19.222222,208.962963,1.814815
3,--8LPVSo5i0Oo61X01sV9A,25.250000,2095.750000,0.500000,7.500000,0.000000
4,--9QQLMTbFzLJ_oT-ON3Xw,52.454545,1804.636364,1.000000,34.636364,0.090909


In [26]:
businesses.describe()

,alcohol?,good_for_kids,has_bike_parking,has_wifi,is_open,latitude,longitude,price_range,review_count,stars,take_reservations,takes_credit_cards
count,188593.000000,188593.000000,188593.000000,188593.000000,188593.000000,188587.000000,188587.000000,188593.000000,188593.000000,188593.000000,188593.000000,188593.000000
mean,0.140610,0.279029,0.350692,0.134968,0.830391,38.506793,-97.490873,1.035855,31.797310,3.631550,0.106086,0.700243
std,0.347619,0.448523,0.477188,0.341690,0.375290,5.122684,17.693360,1.037985,104.124212,1.016783,0.307948,0.458152
min,0.000000,0.000000,0.000000,0.000000,0.000000,-71.753941,-180.000000,0.000000,3.000000,1.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1.000000,33.630878,-112.279276,0.000000,4.000000,3.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,1.000000,36.143595,-111.777460,1.000000,9.000000,3.500000,0.000000,1.000000
75%,0.000000,1.000000,1.000000,0.000000,1.000000,43.593106,-79.982958,2.000000,24.000000,4.500000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,85.051129,115.086769,4.000000,7968.000000,5.000000,1.000000,1.000000


In [27]:
checkins.describe()

,weekday_checkins,weekend_checkins
count,157075.000000,157075.000000
mean,54.491873,59.567551
std,361.550139,363.523671
min,0.000000,0.000000
25%,2.000000,2.000000
50%,8.000000,7.000000
75%,30.000000,31.000000
max,73830.000000,64647.000000


In [28]:
photos.describe()

,average_caption_length,number_pics
count,32976.000000,32976.000000
mean,16.195511,8.521106
std,19.515889,18.780734
min,0.000000,1.000000
25%,0.000000,1.000000
50%,11.333333,4.000000
75%,23.875000,9.000000
max,140.000000,1150.000000


In [29]:
reviews.describe()

,average_review_age,average_review_length,average_review_sentiment,number_funny_votes,number_cool_votes,number_useful_votes
count,188593.000000,188593.000000,188593.000000,188593.000000,188593.000000,188593.000000
mean,1175.501021,596.463567,0.554935,15.617091,18.495973,43.515279
std,659.377141,267.858613,0.345654,109.837911,82.337083,170.623422
min,71.555556,62.400000,-0.995200,0.000000,0.000000,0.000000
25%,714.666667,421.750000,0.373775,0.000000,1.000000,4.000000
50%,1075.741627,546.000000,0.641700,3.000000,3.000000,11.000000
75%,1498.000000,708.561475,0.819559,10.000000,11.000000,34.000000
max,4727.333333,4229.000000,0.996575,36822.000000,6572.000000,38357.000000


In [30]:
users.describe()

,average_number_friends,average_days_on_yelp,average_number_fans,average_review_count,average_number_years_elite
count,188593.000000,188593.000000,188593.000000,188593.000000,188593.000000
mean,105.132000,2005.367009,11.590148,122.110660,0.923313
std,162.653680,554.174540,25.901801,208.814471,1.109289
min,1.000000,76.000000,0.000000,0.666667,0.000000
25%,26.666667,1647.000000,0.666667,21.000000,0.000000
50%,59.384615,1957.150000,3.583333,62.142857,0.583333
75%,117.666667,2312.238095,11.555556,143.454545,1.400000
max,4219.000000,4860.000000,1174.666667,6335.000000,10.666667


In [31]:
tips.describe()

,average_tip_length,number_tips
count,121526.000000,121526.000000
mean,70.832832,9.753863
std,51.687652,31.992793
min,1.000000,1.000000
25%,44.500000,1.000000
50%,60.000000,3.000000
75%,79.818182,7.000000
max,500.000000,3581.000000


Information we have until now: 

- Main table Businesses has all the details about each business (188,593) that will give us information about why they could have their ratings (target, features) 
- Reviews is the table that gives us the information about the perceived value of the business 
- Users is an already grouped table by business analyzing the reviewers for each business. Will be useful for analyzing the weight of the rating for the business (users with higher time spent on yelp, higher/lower number of reviews, if they have reputation friends and fans, etc)
- Tips will give us extra information about value perception from the user by business
- Checkins is useful to see there is a relation between the user's behaviour and the businesses' rating. 
- All the tables are already grouped by business, so they share the business id which will be our key to merge and combine tables if we need to. We don't need to do any previous grouping or cleaning for each one as we can do this after merging
- Checkins, tips and photos don't have the same length as the other tables so this means that not all business have this information (this means when we merge we will have some null values that we will have to fix)

Next steps: merge all the tables in one based on business id to check for correlation between features, clean table from null values and then we are okay to start the analysis


In [32]:
df = pd.merge (businesses, reviews, how="left", on="business_id")

In [35]:
#check that the n of businesses is the same as the original table
print(len(df))

188593


In [36]:
#merge the rest of tables

df = pd.merge(df, users, how='left', on='business_id')
df = pd.merge(df, checkins, how='left', on='business_id')
df = pd.merge(df, tips, how='left', on='business_id')
df = pd.merge(df, photos, how='left', on='business_id')

In [38]:
#check again to make sure
print(len(df))

188593


In [ ]:
#check if we can remove some columns first (features that we don't need to do the analysis). Stars is the target and the rest have to be features

df.head()

,address,alcohol?,attributes,business_id,categories,city,good_for_kids,has_bike_parking,has_wifi,hours,...,average_number_fans,average_review_count,average_number_years_elite,time,weekday_checkins,weekend_checkins,average_tip_length,number_tips,average_caption_length,number_pics
0,1314 44 Avenue NE,0,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,1,0,0,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",...,5.375000,114.875000,1.125000,"{'Fri-0': 1, 'Sun-0': 3, 'Thu-0': 1, 'Fri-1': ...",11.0,18.0,38.0,6.0,NaN,NaN
1,,0,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,1,0,0,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",...,2.666667,19.666667,0.666667,{'Sun-1': 1},0.0,1.0,66.0,1.0,29.0,1.0
2,1335 rue Beaubien E,1,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,1,1,1,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",...,1.000000,24.600000,0.600000,"{'Mon-0': 1, 'Sun-0': 1, 'Fri-1': 1, 'Sun-1': ...",2.0,5.0,32.0,1.0,NaN,NaN
3,211 W Monroe St,0,None,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Phoenix,0,0,0,None,...,2.375000,68.000000,0.250000,{'Wed-18': 1},1.0,0.0,NaN,NaN,NaN,NaN
4,2005 Alyth Place SE,0,{'BusinessAcceptsCreditCards': 'True'},8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,0,0,0,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",...,26.750000,528.250000,4.250000,"{'Thu-14': 1, 'Thu-20': 1}",2.0,0.0,NaN,NaN,NaN,NaN


In [46]:
df.columns

Index(['address', 'alcohol?', 'attributes', 'business_id', 'categories',
       'city', 'good_for_kids', 'has_bike_parking', 'has_wifi', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'neighborhood',
       'postal_code', 'price_range', 'review_count', 'stars', 'state',
       'take_reservations', 'takes_credit_cards', 'average_review_age',
       'average_review_length', 'average_review_sentiment',
       'number_funny_votes', 'number_cool_votes', 'number_useful_votes',
       'average_number_friends', 'average_days_on_yelp', 'average_number_fans',
       'average_review_count', 'average_number_years_elite', 'time',
       'weekday_checkins', 'weekend_checkins', 'average_tip_length',
       'number_tips', 'average_caption_length', 'number_pics'],
      dtype='object')

In [47]:
features_to_remove = ['address','attributes','business_id','categories','city','hours','is_open','latitude','longitude','name','neighborhood','postal_code','state','time']
df.drop(labels=features_to_remove, axis=1, inplace=True)

In [48]:
df.isna().sum()

alcohol?                           0
good_for_kids                      0
has_bike_parking                   0
has_wifi                           0
price_range                        0
review_count                       0
stars                              0
take_reservations                  0
takes_credit_cards                 0
average_review_age                 0
average_review_length              0
average_review_sentiment           0
number_funny_votes                 0
number_cool_votes                  0
number_useful_votes                0
average_number_friends             0
average_days_on_yelp               0
average_number_fans                0
average_review_count               0
average_number_years_elite         0
weekday_checkins               31518
weekend_checkins               31518
average_tip_length             67067
number_tips                    67067
average_caption_length        155617
number_pics                   155617
dtype: int64

In [49]:
df.isna().any()

alcohol?                      False
good_for_kids                 False
has_bike_parking              False
has_wifi                      False
price_range                   False
review_count                  False
stars                         False
take_reservations             False
takes_credit_cards            False
average_review_age            False
average_review_length         False
average_review_sentiment      False
number_funny_votes            False
number_cool_votes             False
number_useful_votes           False
average_number_friends        False
average_days_on_yelp          False
average_number_fans           False
average_review_count          False
average_number_years_elite    False
weekday_checkins               True
weekend_checkins               True
average_tip_length             True
number_tips                    True
average_caption_length         True
number_pics                    True
dtype: bool

In [ ]:
#The rows that have null values are the ones from tips, photos and checkins as we predicted before, so we can substitute them for 0 values so we don't lose all the row information from the other features

df.fillna({'weekday_checkins':0,
           'weekend_checkins':0,
           'average_tip_length':0,
           'number_tips':0,
           'average_caption_length':0,
           'number_pics':0},
          inplace=True)

#could also do df.fillna(0, inplace=True) which replaces every NaN with 0


In [51]:
#check null values again

df.isna().any()

alcohol?                      False
good_for_kids                 False
has_bike_parking              False
has_wifi                      False
price_range                   False
review_count                  False
stars                         False
take_reservations             False
takes_credit_cards            False
average_review_age            False
average_review_length         False
average_review_sentiment      False
number_funny_votes            False
number_cool_votes             False
number_useful_votes           False
average_number_friends        False
average_days_on_yelp          False
average_number_fans           False
average_review_count          False
average_number_years_elite    False
weekday_checkins              False
weekend_checkins              False
average_tip_length            False
number_tips                   False
average_caption_length        False
number_pics                   False
dtype: bool

In [52]:
df.head()

,alcohol?,good_for_kids,has_bike_parking,has_wifi,price_range,review_count,stars,take_reservations,takes_credit_cards,average_review_age,...,average_days_on_yelp,average_number_fans,average_review_count,average_number_years_elite,weekday_checkins,weekend_checkins,average_tip_length,number_tips,average_caption_length,number_pics
0,0,1,0,0,2,24,4.0,1,1,618.250000,...,1809.375000,5.375000,114.875000,1.125000,11.0,18.0,38.0,6.0,0.0,0.0
1,0,1,0,0,2,3,4.5,0,1,371.666667,...,970.333333,2.666667,19.666667,0.666667,0.0,1.0,66.0,1.0,29.0,1.0
2,1,1,1,1,2,5,4.0,1,0,1106.200000,...,1499.600000,1.000000,24.600000,0.600000,2.0,5.0,32.0,1.0,0.0,0.0
3,0,0,0,0,0,8,1.5,0,0,398.500000,...,1626.500000,2.375000,68.000000,0.250000,1.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,4,2.0,0,1,1412.750000,...,2570.000000,26.750000,528.250000,4.250000,2.0,0.0,0.0,0.0,0.0,0.0


## EDA